# Analysing the best performing model using Captum to understand the decision making process from the model

##### This notebook follows the official tutorial for BERT models from Captum as can be found in this link: https://captum.ai/tutorials/Bert_SQUAD_Interpret

In [45]:
import pandas as pd
import torch
from captum.attr import LayerIntegratedGradients, visualization as viz
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig

# Prepare data and load model

In [46]:
# Load prediction and test data
model_pred_df = pd.read_csv(
    '../2. models/predictions/nickmuchi-sec-bert-finetuned-finance-classification_False_FinancialStrengthText_MAMConfig_2024-04-25_predictions.csv')
test_df = pd.read_csv('../1. data/final/test.csv')

# Identify the indices of texts with the three lowest and highest prediction values
lowest_indices = model_pred_df['1'].nsmallest(3).index
highest_indices = model_pred_df['1'].nlargest(3).index

# Retrieve the actual texts corresponding to these indices for analysis
lowest_texts = test_df.loc[lowest_indices, 'FinancialStrengthText']
highest_texts = test_df.loc[highest_indices, 'FinancialStrengthText']

In [47]:
# Set up the computation device for PyTorch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [48]:
# Load the pre-trained model and its configuration from a checkpoint
checkpoint_path = '../results/llm_fine-tine_MAMConfig/nickmuchi-sec-bert-finetuned-finance-classification/checkpoint-276'
config = AutoConfig.from_pretrained(checkpoint_path)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path, config=config)
model.to(device)
model.eval()  # Set model to evaluation mode
model.zero_grad()  # Zero gradients to ready for new computation

In [49]:
# Load tokenizer to convert text into format suitable for the model
tokenizer = AutoTokenizer.from_pretrained('nickmuchi/sec-bert-finetuned-finance-classification')

# Define special token IDs that will be used to format input data properly
ref_token_id = tokenizer.pad_token_id  # Padding token for reference sequences
sep_token_id = tokenizer.sep_token_id  # Separator token (often used in BERT models)
cls_token_id = tokenizer.cls_token_id  # Classifier token (marks start of sequence in BERT)

# Captum part

#### Helper functions

In [50]:
def predict(inputs):
    """Make a forward pass through the model to obtain predictions."""
    return model(inputs)[0]

def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):
    """Prepare input and reference pairs required for Captum's attribution analysis."""
    text_ids = tokenizer.encode(text, add_special_tokens=False)
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]
    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device)


def custom_forward(inputs):
    """Custom forward function to extract specific outputs for analysis with Captum."""
    preds = predict(inputs)
    return torch.softmax(preds, dim=1)[0][0].unsqueeze(-1)


def summarize_attributions(attributions):
    """Normalize and summarize attributions across the input tokens."""
    attributions = attributions.sum(dim=-1).squeeze(0)
    return attributions / torch.norm(attributions)

### Analyze the text

In [51]:
def analyze_texts(texts):
    """Analyze given texts to visualize model attributions and their impact on predictions."""
    lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

    for text in texts:
        input_ids, ref_input_ids = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
        score = predict(input_ids)

        attributions, delta = lig.attribute(inputs=input_ids, baselines=ref_input_ids, return_convergence_delta=True)
        attributions_sum = summarize_attributions(attributions)

        indices = input_ids[0].detach().tolist()
        all_tokens = tokenizer.convert_ids_to_tokens(indices)

        # Prepare data for visualization
        score_vis = viz.VisualizationDataRecord(
            attributions_sum,
            torch.softmax(score, dim=1)[0][0],
            torch.argmax(torch.softmax(score, dim=1)[0]),
            0,
            text,
            attributions_sum.sum(),
            all_tokens,
            delta)

        print('\033[1m', 'Visualization For Score', '\033[0m')
        viz.visualize_text([score_vis])

### Perform analysis on texts with the lowest and highest model predictions

In [52]:
print("Analyzing lowest texts:")
analyze_texts(lowest_texts)

Analyzing lowest texts:
 Visualization For Score 


 Visualization For Score 


 Visualization For Score 


In [53]:
print("Analyzing highest texts:")
analyze_texts(highest_texts)

Analyzing highest texts:
 Visualization For Score 


 Visualization For Score 


 Visualization For Score 


### Manually add and analyze a specific custom text

In [55]:
custom_text = [
    "We believe Under Armour has enough liquidity to get through the current economic environment. Prior to the outbreak of COVID-19, the firms long-term debt consisted only of $593 million in 3.25% senior unsecured notes that mature in 2026. Then, in May 2020, Under Armour completed an offering of $500 million in 1.5% convertible senior notes that mature in 2024. However, as this additional funding has proven to be unnecessary, the firm has already paid down more than 80% of this convertible debt. Even after these debt repayments, at the end of June 2023, the firm had $704 million in cash as well as $1.1 billion in borrowing capacity under its revolver. We expect Under Armour will operate in a net cash position for the foreseeable future. Under Armours free cash flow to the firm has averaged about $300 million per year over the past five years but has been erratic. We forecast free cash flow at only $117 million in fiscal 2024, down from $332 million in the prior year, due to investments related to its Protect This House 3 plan and lower earnings due to low wholesale orders.Although Under Armour does not pay dividends, it recently authorized its first share buyback program. The firm repurchased $300 million in shares in February 2022, and then $125 million in buybacks in fiscal 2023. Moreover, its restructuring has reduced base operating expenses by about $200 million, and we forecast its capital expenditures will remain low at 3%-4% of sales in the long term. Under Armour may use some of its free cash flow for acquisitions, but we do not forecast acquisitions due to the uncertainty concerning timing and size. Although its growth has been largely organic, the firm acquired three fitness apps for a combined $710 million in past years as part of a strategy that has been mostly abandoned. It has also made some smaller investments, such as $39.2 million in its Japanese licensee, Dome, in 2018. Under Armour later had to write down this investment."]
analyze_texts(custom_text)

 Visualization For Score 
